#  Alignment to Human1:

Sequence of runing for CI

1.  Reactions: update GPR  and ec-code lists
2.  Genes: annotate new genes comming from updated GPR list
3.  Metabolites: annotate metabolites

# Dependencies and Libraries

In [22]:
# Install packages if not already available
import sys
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# External packages
for pkg in ["cobra", "pandas", "memote"]:
    try:
        __import__(pkg) 
    except ImportError:
        install(pkg)


import cobra
import re
from pprint import pprint
import pandas as pd
import memote
import gc

import ssl #certificates needed to access KEGG API 
ssl._create_default_https_context = ssl._create_unverified_context #create the certificate

# delete later

In [23]:
def visualization_model_annotations(model, component):
    """"
    This function retrieves all elements inside 'annotations' as dictionary
    
    -model: COBRApy model object (e.g., mitocore)
    -component: str  ('reactions', 'metabolites', 'genes')
    """
    #read SBML model
    mitocore= model
    for comp in mitocore.__getattribute__(component):
        comp_complete= comp.__dict__
        print(comp_complete )

def visualization_gpr(mitocore ,name,timeline):
    """ function to extract number of reactions with GPR and without GPR in a metabolic SBML model
    
    -model: str, path to the SBML model
    -name: str , name of the model
    -timeline: str, stand of the model for naming (e.g. prior, after)
    """

    model= mitocore
    has_gpr=[]
    no_gpr=[]

    for reaction in model.reactions:
        if reaction.gene_reaction_rule.strip():
            has_gpr.append(reaction)
        else:
            no_gpr.append(reaction)

    print(f"{name} reactions with GPR {timeline}:", len(has_gpr))
    print(f"{name}_reactions empty GPR {timeline}:", len(no_gpr))

# Workflow Functions to:
1. Extract ids from model  
2. Map them to a target database in a mapping file
3. Add them to current model
4. Get attribute (not inside of attribute 'annotation', e.g. GPR rule)
5. Add GPR rule to the model and complete if already present.

In [24]:
# 1. extract ids from model  
def get_model_ids_for_databases(model, component_type, dict, databases):
    """
    Extract ids from model annotations for various databases across a specified component type.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - dict: dictionary where database ids are located ('annotations', 'notes')
    - databases: list of str, e.g., ['kegg', 'bigg', 'metanetx', 'uniprot'].
    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': []}
    for db in databases: 
        ids_dict[db] = []

    # Access model core-component (reactions, genes, metabolites)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)

        for db in databases:
            value = None
            annotation = getattr(comp, dict, {})

            # look for different naming conventions
            for key in [db, 
                        f"{db}.reaction", 
                        f"{db}.compound", 
                        f"{db}.metabolite", 
                        f"{db}.chemical", 
                        f"{db}.genes",
                        f"{db}.gene",
                        f"{db} id",
                        f"{db.upper()} id",
                        ]:
                if key in annotation:
                    value = annotation[key]
                    break

            ids_dict[db].append(value)

    return pd.DataFrame(ids_dict)

# 2. map them to a target database in a mapping file
def map_ids_to_db(id_df, input_df, id_col_input, id_col_map, target_db_col, sep_map_file):
    """
    Map ids from a column in a dataframe to another column of a dataframe and add the target identifier.
    
    Parameters:
    - id_df: input Data frame with model_ids and ids to a specific database to be mapped.
    -input_df: dataframe for mapping ids to the same database as id_df and target database
    -id_col_input: column in id_df that contains ids to be mapped.
    -id_col_map: the column in mapping_file that contains the ids to be mapped.
    -target_db_col: the column in mapping_file that contains the target database ids to be extracted.
    -sep_map_file: the separator used in the mapping file (default is ',').
    
    Returns:
    - DataFrame with model ids and extracted target database ids.
    """
    import pandas as pd
    ids_dict = {'model_id': [],target_db_col: []}

    #load csv mapping file
    mapping_df = input_df

    # iterate over ids 
    for index, row in id_df.iterrows():
        id = row[id_col_input]

        # check for model ids are present in mapping file 
        match= mapping_df[mapping_df[id_col_map] == id]
        # if id model existing extract target id (target database)
        if not match.empty:
            target_id= match[target_db_col].values[0]
            print(f"Match found: {id} -> {target_id}")
        # if id model not found extract 'None' value
        else:
            target_id= None
            print(f"No match found for {id}, using default value None")

        #collect target ids into ids dictionary
        ids_dict['model_id'].append(row['model_id'])
        ids_dict[target_db_col].append(target_id)
    
    #return dictionary as dataframe
    ids_dict= pd.DataFrame(ids_dict)
    return ids_dict

# 3. add them to current model (used in manual curation because of stringfication of lists when converting to .csv)
def add_ids_to_model(model, df, component_type, database):
    """
    Add ids from a DataFrame to the model annotations for a specified component type.

    Parameters:
    - model: COBRApy model object (e.g., mitocore)
    - df: DataFrame with model_id and database id columns.
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - database: str, the name of the database column in df to map (e.g., 'kegg', 'bigg')
    
    Returns:
    - Updated model with new annotations added (only if not already present).
    """
    import ast
     
    components = getattr(model, component_type)

    # iterate over ids (rows)
    for index, row in df.iterrows():
        model_id = row['model_id']
        db_value = row[database]

        #check that database id in dataframe exists  (is not NaN or 'None')
        if pd.notna(db_value) and db_value != 'None':
            if isinstance(db_value, str) and db_value.startswith("[") and db_value.endswith("]"):
                try: 
                    db_value= ast.literal_eval(db_value)
                except Exception as e:
                    print(f"Error parsing {db_value}: {e}")
                    continue

            for comp in components:
                if comp.id == model_id:
                        if isinstance(db_value, list) and len(db_value) == 1:
                            db_value = db_value[0]
                        # Add only if not in annotation yet
                        comp.annotation[database] = db_value
                        print(f"Adding {db_value} class {type(db_value)} to {model_id}")
                        break 
    return model

# 4. get attribute (not inside of attribute 'annotation', e.g.GPR rule)
def get_model_ids_for_attr(model, component_type, attr_name):
    """
    Extract attribute in the model in the target core-component.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: Core component of GEMs, str, one of 'reactions', 'metabolites', 'genes'
    - attr_name: name of the dictionary where target object is located ('notes', '_gpr')

    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # dictionary to collect ids and attributes
    ids_dict = {'model_id': [], attr_name:[]}

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    # iterate over components and extract attribute
    for comp in components:
        #add model id and attribute to dictionary
        ids_dict['model_id'].append(comp.id)
        value = comp.__dict__.get(attr_name, None) #None in case no attribute present
        ids_dict[attr_name].append(value) 

    #returns dataframe with attributes
    return pd.DataFrame(ids_dict)

# 5. add GPR rule to the model and complete if already present
def add_gpr_to_model(model, df, component_type, gpr_header_in_df):
    """
    check if attribute GPR is empty in the model or already present,
    Add ids from a dataFrame to the gpr attribute if empty or append to existing GPR rule.

    Parameters:
    - model: COBRApy model object (e.g., mitocore)
    - df: DataFrame with model_id and formulas column.
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - gpr_header_in_df: str, the name of the database column in df to map (e.g., '_gpr')
    
    Returns:
    - Updated model with new annotations added (only if not already present).
    """
    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type )

    # iterate over ids (rows)
    for index, row in df.iterrows():
        model_id = row['model_id']
        new_rule = str(row[gpr_header_in_df]).strip() if pd.notnull(row[gpr_header_in_df]) else ''

        # find matching component based on model_id
        for comp in components:
            if comp.id == model_id:
                existing_rule = comp.gene_reaction_rule.strip()
                #check if GPR is empty)
                if existing_rule == '' and new_rule != '':
                    comp.gene_reaction_rule = new_rule
                    print(f"Added GPR: {comp.id} -> {new_rule}")
                #check if GPR already exists and append new rule with 'or' (does not avoid duplicates, this issue is corrected later)
                elif existing_rule != '' and new_rule != '':
                    comp.gene_reaction_rule = existing_rule + ' or ' + new_rule
                    print(f"GPR already exists for {comp.id}: {existing_rule}. New rule {new_rule} added.")
                break  # Stop once matched
    return model

In [25]:
# new function for add ids to model in automatic pipeline
def add_ids_to_model(model, df, component_type, database):
    """
    Add ids from a DataFrame to the model annotations for a specified component type.
    new defined in automation pipeline to handle not stringtified lists in dataframes insted of .csv stringtified lists 

    Parameters:
    - model: COBRApy model object (e.g., mitocore)
    - df: DataFrame with model_id and database id columns.
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - database: str, the name of the database column in df to map (e.g., 'kegg', 'bigg')
    
    Returns:
    - Updated model with new annotations added (only if not already present).
    """
    import ast
    components = getattr(model, component_type)

    for index, row in df.iterrows():
        model_id = row['model_id']
        db_value = row[database]

        # Skip missing or None values safely
        if db_value is None or (isinstance(db_value, float) and pd.isna(db_value)):
            continue

        # Convert stringified lists (from CSVs) into real lists
        if isinstance(db_value, str) and db_value.startswith("[") and db_value.endswith("]"):
            try:
                db_value = ast.literal_eval(db_value)
            except Exception as e:
                print(f"Error parsing {db_value}: {e}")
                continue

        # Now db_value is either a string or a real list
        for comp in components:
            if comp.id == model_id:
                if isinstance(db_value, list) and len(db_value) == 1:
                    db_value = db_value[0]
                comp.annotation[database] = db_value
                print(f"Added {db_value} (type {type(db_value)}) to {model_id}")
                break

    return model

# Cleaning-up false positive entries
During the curation process, placeholder entries such as N/A, nan, or None may have been introduced into the model. These values must be removed.
Note: MEMOTE testing tool interprets them as valid (non-empty) entries. As a result, MEMOTE classifies them as positive identifiers, which can give misleading results

In [26]:
# --Function to clean up N/A, None, nan... Values---
def clean_invalid_annotations(model, invalid_values={'N/A', 'None', 'nan', 'NaN', ''}):
    """
    Remove invalid placeholder values from model annotations across reactions, metabolites, and genes to avoid false positives in MEMOTE test.
    
    -model: COBRApy model object (already read with COBRApy, e.g. Mitocore)
    -invalid_values: dict, keys to be removed
    """
    #acces model's objects core-components 
    for component in model.reactions + model.metabolites + model.genes:
        #collect keys to remove
        keys_to_remove = []
        for key, val in component.annotation.items():
            if isinstance(val, (str, type(None))):
                #check if value inside key is in invalid dict
                if str(val).strip() in invalid_values:
                    keys_to_remove.append(key)
        for key in keys_to_remove:
            del component.annotation[key]
    # return model with changes (does not save model as new SBML file)
    return model

# MetaNetx file processing functions

There are two informations necessary from MetaNetX file:
1. MetaNetX identifier
2. ec-code

In [27]:
# 1.MetaNetX identifier
def mnx_processing_file_mnx(mnx_file,db_map,component,start_patterns):
    """Process the MNX mapping file to extract relevant columns and save it as a CSV.

    Parameters:
    - mnx_file: metanetx _xref.tsv file to process
    - db_map: reference ids to map for ['bigg', 'kegg'...]
    - component: either 'reactions' or 'metabolites' can be tracked in metanetx database.
    - start_patterns: Tupple of patterns to look for in file for id processing:
        e.g. ( "kegg.compound:", "keggC:") or ("bigg.metabolite:", "biggM:")
    
    Returns:
    - DataFrame with component id and extracted database ids.

    """ 
    #dictionary to collect database ids and respective metanetx ids
    mapping = {db_map: [], f"metanetx.{component}": []}

    #open and read metanetx mapping file
    with open(mnx_file, "r", encoding="utf-8") as file:
        # iterate over lines in file
        for line in file:
            #check for comments (#) and empty lines
            if line.startswith('#') or not line.strip():
                continue
            
            # split line by tab and check if has at least 3 columns
            parts = line.strip().split('\t')
            if len(parts) < 3:
                continue  # Skip malformed lines
            
            # mnx id is in 2°nd part out of 3 parts
            source = parts[0]
            target_mnx_id = parts[1]
            rest = parts[2]

            #generalize starting patterns to look for   KEGG or BIGG
            if source.startswith(start_patterns):
                #recon_id is a generic name for database id
                recon_id = source.split(":", 1)[1].strip()
                mnx_id = target_mnx_id.strip()
                mapping[db_map].append(recon_id)
                mapping[f"metanetx.{component}"].append(mnx_id)

    #return dataframe of collecting dictionary
    metanetx_react = pd.DataFrame.from_dict(mapping, orient="columns")
    return metanetx_react
# 2.ec-code
def mnx_processing_file_eccode(mnx_file, db_map):
    """Process the MNX mapping file to extract ec-codes and database ids.

    Parameters:
    - mnx_file: Path to the MetaNetX file with reaction or metabolite data
    - db_map: Database name (e.g., 'bigg', 'kegg') to store as a column

    Returns:
    - DataFrame with db_map and EC code columns (EC is a list if >1, else string)
    """
    # dictionary to collect database ids and respective ec-code
    mapping = {db_map: [], 'ec-code': []}
    # Regular expression pattern to match ec-code
    ec_pattern = re.compile(r"\b\d+\.\d+\.\d+(?:\.\d+|\.n)?\b")
    
    #open and read metanetx mapping file
    with open(mnx_file, "r", encoding="utf-8") as file:
        # iterate over lines in file
        for line in file:
            if line.startswith('#') or not line.strip():
                continue

            # split line by tab and check if has at least 4 columns
            parts = line.strip().split('\t')
            if len(parts) < 4:
                continue  # Skip malformed lines

            # ec-code is in 4°nd part out of 4 parts
            mnx_id = parts[0].strip()
            ec_codes = parts[3].strip()


            if ec_pattern.search(ec_codes):
                codes_patter= re.compile(r'^\d+\.\d+\.\d+\.\d+$')
                codes = [code.strip() for code in ec_codes.split(';') if code.strip()]
                codes = [code for code in codes if codes_patter.match(code)]
                # Convert to string if only one ec-code
                formatted_ec = codes[0] if len(codes) == 1 else codes
                mapping[db_map].append(mnx_id)
                mapping['ec-code'].append(formatted_ec)
                print(f"{formatted_ec} is  {type(formatted_ec)}")

    # return dataframe of collecting dictionary for ec-codes and respective mnx ids
    df = pd.DataFrame(mapping)
    return df

# Load Models
- MitoCore
- Human1

mitocore= cobra.io.read_sbml_model("Models/MitoCore.xml")
human1= cobra.io.read_sbml_model('Models/Human-GEM.xml')

In [ ]:
mitocore= cobra.io.read_sbml_model("Models/MitoCore.xml")
human1= cobra.io.read_sbml_model('Models/Human-GEM.xml')

# ----------------------------------REACTIONS CHECKPOINT-----------------------------------------


# Add MetaNetX ids to model

Worflow:
1. Load last SBML model and extract database ids into dataframe
2. Process MNX tsv file (function 1.) and save to csv
3. Get ids from the model [KEGG, BiGG]
4. Map model ids to metanetx processed  csv file using [BiGG] --> Why not using KEGG too??
5. Add MNX ids to the model


In [29]:

# 2. Process mnx file (MNX processing function 1.)
mnx_reactions= mnx_processing_file_mnx('Files/reac_xref.tsv','Recon2','reaction',("bigg.reaction:", "biggR:"))
print('processes MNX reaction file')
print(mnx_reactions)

# 3. get kegg and bigg ids from model
print('geting ids for all possible databases ')
df_react = get_model_ids_for_databases(mitocore, 'reactions', 'notes', ['KEGG', 'Recon2'])
print(df_react)

# 4. map model ids (bigg) to metanetx processed  csv file
print('Mapping  metabolites to metanetx.')
df_mnx_react = map_ids_to_db(df_react, mnx_reactions,'Recon2', 'Recon2', 'metanetx.reaction', ',')
print('MNX metabolites \n',df_mnx_react)

# 5. Add mnx ids to the model
print('adding MNX ids to the model')
mnx_react = add_ids_to_model(mitocore, df_mnx_react, 'reactions','metanetx.reaction' )




processes MNX reaction file
             Recon2 metanetx.reaction
0            CRBNTD             EMPTY
1          DNADRAIN             EMPTY
2           H2CO3D2             EMPTY
3          H2CO3D2m             EMPTY
4          HMR_5409             EMPTY
...             ...               ...
112679  R_GALNACT3g         MNXR99998
112680    GALNACT4g         MNXR99999
112681  R_GALNACT4g         MNXR99999
112682    GALNACT4g         MNXR99999
112683  R_GALNACT4g         MNXR99999

[112684 rows x 2 columns]
geting ids for all possible databases 
       model_id  KEGG      Recon2
0      EX_2hb_e  None        None
1       EX_ac_e  None        None
2     EX_acac_e  None        None
3      EX_akg_e  None        None
4    EX_ala_B_e  None        None
..          ...   ...         ...
550         COt  None         COt
551         NOt  None         NOt
552  PCHOLHSTDe  None  PCHOLHSTDe
553        PSt3  None        PSt3
554         PEt  None         PEt

[555 rows x 3 columns]
Mapping  metabolit

# Add new Human1 ids to model 
### based on upgraded  MetaNetX ids 
- MetaNetX is the only id updated during CI so no need to test for KEGG or BiGG 

Worflow:
1. Load last SBML model and extract database ids into dataframe
2. Map reactions to Human1 in .tsv file and rename naming for Human1
3. Add Human1 ids from dataframe to model.


In [30]:

print('geting ids for all MNX')
df_react = get_model_ids_for_databases(mitocore, 'reactions', '_annotation', ['metanetx'])
print(df_react)

# 2. map metanetx ids to Human1 mapping file
human1_react= pd.read_csv('Files/reactions.tsv', sep='\t')
print('Mapping mnx reactions to human1')
df_human1_react = map_ids_to_db(df_react, human1_react,'metanetx', 'rxnMetaNetXID', 'rxns', '\t')
print('human1 reactions \n',df_human1_react)
df_human1_react = df_human1_react.rename(columns={'model_id': 'model_id', 'rxns': 'Human1'})
print(df_human1_react)

# 3. add Human1 ids to model
print('adding human1 ids to the model')
human1_react = add_ids_to_model(mitocore, df_human1_react, 'reactions','Human1')

geting ids for all MNX
       model_id    metanetx
0      EX_2hb_e        None
1       EX_ac_e        None
2     EX_acac_e        None
3      EX_akg_e        None
4    EX_ala_B_e        None
..          ...         ...
550         COt  MNXR153251
551         NOt  MNXR191149
552  PCHOLHSTDe  MNXR102406
553        PSt3  MNXR192024
554         PEt  MNXR102505

[555 rows x 2 columns]
Mapping mnx reactions to human1
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using defa

In [31]:
visualization_mito_annotations= visualization_model_annotations(mitocore,'reactions')

{'_id': 'EX_2hb_e', 'name': 'EX_2hb_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite 2hb_e at 0x16874f050>: -1.0}, '_model': <Model S1SBMLmodel at 0x165692810>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_ac_e', 'name': 'EX_ac_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite ac_e at 0x168f6d6d0>: -1.0}, '_model': <Model S1SBMLmodel at 0x165692810>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_acac_e', 'name': 'EX_acac_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite acac_e at 0x168ebf950>: -1.0}, '_model': <Model S1SBMLmodel at 0x165692810>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_akg_e', 'name': 'EX_akg_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '',

# --------Checkpoint MetaNetX, Human1 ids updated----

In [32]:
## --Visualization of GPR rules in Mitocore prior--
mito=visualization_gpr(mitocore,'mitocore', 'prior')

mitocore reactions with GPR prior: 353
mitocore_reactions empty GPR prior: 202


# Align Mitocore GPR list to Human1

Worflow:

2. Extract Human1 ids from Mitocore into dataframe.
3. Extract GPRs from models
4. Merge Dataframes 
5. Parse GPRs present in both models
6. Add GPRs (risk of duplicates, cleaned later) and run Memote test


In [33]:

# 2. extract Human1 ids in mitocore
human1_mitocore=get_model_ids_for_databases(mitocore, 'reactions', '_annotation', ['Human1'])
print(human1_mitocore)

# 3. extract GPR rules from models
gpr_mitocore= get_model_ids_for_attr(mitocore, 'reactions', '_gpr')
gpr_human1= get_model_ids_for_attr(human1, 'reactions', '_gpr')

# 4. merge mitocore dfs to have Human1 ids and GPR rules.
gpr_mitocore = gpr_mitocore.merge(human1_mitocore, on='model_id')
print('df with _gpr and model ids in Mitocore', gpr_mitocore)

# 5. df with gpr from human1 mapped to mitocore
gpr_parsed=map_ids_to_db(gpr_mitocore, gpr_human1, 'Human1', 'model_id', '_gpr', ',')

# 6. add gpr from human1 to mitocore and Memote report after adding GPRs
gpr_add=add_gpr_to_model(mitocore, gpr_parsed, 'reactions', '_gpr')

       model_id    Human1
0      EX_2hb_e      None
1       EX_ac_e      None
2     EX_acac_e      None
3      EX_akg_e      None
4    EX_ala_B_e      None
..          ...       ...
550         COt      None
551         NOt      None
552  PCHOLHSTDe  MAR07158
553        PSt3      None
554         PEt  MAR08528

[555 rows x 2 columns]
df with _gpr and model ids in Mitocore        model_id _gpr    Human1
0      EX_2hb_e           None
1       EX_ac_e           None
2     EX_acac_e           None
3      EX_akg_e           None
4    EX_ala_B_e           None
..          ...  ...       ...
550         COt           None
551         NOt           None
552  PCHOLHSTDe       MAR07158
553        PSt3           None
554         PEt       MAR08528

[555 rows x 3 columns]
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, usin

# Cleanning GPRs from duplicates 

the function to add GPRs from Human1 to Memote checks if the reactions has empty GPR and adds it directly. In case there is a GPR present in for the entry, it adds the whole Human1 GPR to Mitocore with 'or' partricle.

## Example:
### -Reaction R1 in Mitocore  --> GPR ['geneA'] 
### -Reaction R1 in Human1 --> GPR ['geneA' or 'geneB']

### after add new GPR to Mitocore:
### - New Reaction R1 in Mitocore --> GPR ['geneA' or 'geneA' or 'geneB']


In [34]:
# Vizualisation of the Duplicates in GPR rules
gpr_mitocore= get_model_ids_for_attr(mitocore, 'reactions', '_gpr')
print(gpr_mitocore[74:150])

              model_id                                               _gpr
74                HEX1  ENSG00000156515 or ENSG00000159399 or ENSG0000...
75              G6PPer                                    ENSG00000131482
76                 PGI                                    ENSG00000105220
77                 PFK                                    ENSG00000152556
78                 FBP                 ENSG00000165140 or ENSG00000130957
..                 ...                                                ...
145    MTPC16_MitoCore                ENSG00000084754 and ENSG00000138029
146  ACADLC14_MitoCore                ENSG00000115361 and ENSG00000171503
147   MECR14C_MitoCore                                    ENSG00000116353
148    MTPC14_MitoCore                ENSG00000084754 and ENSG00000138029
149              r1447                ENSG00000117054 and ENSG00000171503

[76 rows x 2 columns]


# Eliminate duplicates and keep new elements added to GPR
GPR is constructed with 2 conectors for the genes:
#
- 'and' reflects a requirement, genes >1 represent a single element. Built in parenthesis ()
e.g: (geneA and geneB)
- 'or' spearates elements in the GPR, every gene separated by 'or' is a new element. No parenthesis 
e.g: geneA or geneB
# Worflow: 
1. split elements (separator 'or')
2. Collect GPR for every single reaction and eliminate duplicates
3. Split elements inside parenthesis (separator 'and') and randomize them with tuple
4. re-join the GPR by replacing ',' by 'and' and joining single elements with 'or'
5. Create a csv file from the dataframe for manual control.
6. Update GPRs in Mitocore, save new SBML model, and run MEMOTE test


In [35]:

# 1. Split GPR on 'or'  
for index, row in gpr_mitocore.iterrows():
    gpr = row['_gpr']
    elements_in_gpr = str(gpr).split(' or ')

    # 2. Remove duplicates for each reaction with function set()
    unique_parentheses = set() # colect single genes in parenthesis separated by 'and' (ENSG00000115361 and ENSG00000171503) --> ('ENSG00000115361', 'ENSG00000115361')
    unique_single_genes = set()
    cleaned_gprs = [] 
    pattern = r"\((.*?)\)"  # matches content inside parentheses

    # 3. Iterate through each element in the GPR: (parenthesis or single gene) 
    for element in elements_in_gpr:
        element = element.strip()

        # genes in  a parehteses such as: (ENSG00000115361 and ENSG00000171503)
        if element.startswith('(') and 'and' in element:
            match = re.search(pattern, element)
            if match:
                genes = match.group(1).split(' and ')
                #randomize order with tuple: order-independent for comparisson
                genes = tuple(sorted(g.strip() for g in genes)) 
                if genes not in unique_parentheses:
                    # colect single genes in parenthesis separated by 'and' 
                    # (ENSG00000115361 and ENSG00000171503) --> ('ENSG00000115361', 'ENSG00000115361')
                    unique_parentheses.add(genes)
                    #join single genes back with separator 'and' in the cleaned_gprs list
                    cleaned_gprs.append(f"({' and '.join(genes)})")

        # Single gene (not in parentheses)
        elif element not in unique_single_genes:
            unique_single_genes.add(element)
            cleaned_gprs.append(element)

    # 4. update the GPR back to the DataFrame with ' or ' as separator
    gpr_mitocore.at[index, '_gpr'] = ' or '.join(cleaned_gprs)

# 5. update new GPRs from csv to the model, save and test model
for reaction in mitocore.reactions:
    id = reaction.id
    model_gpr = reaction.gene_reaction_rule

    for index, row in gpr_mitocore.iterrows():
        if row['model_id'] == id:  # Use an if statement to check the condition
            new_gpr = row['_gpr']  # Access the '_gpr' column of the current row
            print(f"Updating GPR for {model_gpr} to {new_gpr}")
            reaction.gene_reaction_rule = new_gpr


Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating GPR for  to 
Updating G

In [36]:
## --Visualization of GPR rules in Mitocore prior--
mito=visualization_gpr(mitocore ,'mitocore', 'after updates')

mitocore reactions with GPR after updates: 392
mitocore_reactions empty GPR after updates: 163


# Clean Model from N/A values 

cleans empty entries of None, N/A, and '' values, this values are account into memote as positive entries however these are not valid identifiers

In [37]:
# Clean model from N/A values
mitocore = clean_invalid_annotations(mitocore)

# ---------Checkpoint GPRs updated----

# Update ec-code list in Mitocore

We asume Human1 is a well curated model and align the GPR list if possible to Human1. Do not preserve Mitocore ec-code unless there is no ec-code from Human1 to take over

Worflow:
1. Process MetaNetX file to find ec-codes and save as csv file 
2. Load latest SBML model
3. Extract MetaNetX ids from models
4. Map metanetx ids to ec-codes 
5. Overwrite the ec-codes for each reaction  with new ec-code list from Human1 and Memote test 


In [38]:
# 1.extract ec-codes and corresponding mnx id from metanetx file and create a csv file 
ec_code = mnx_processing_file_eccode('Files/reac_prop.tsv', 'metanetx.reaction')

# 3. extract metanetx ids from model
print('geting ids for all mnx reactions')
df_model_id = get_model_ids_for_databases(mitocore, 'reactions', '_annotation', ['metanetx.reaction'])
print(df_model_id)

# 4. map metanetx ids to ec-codes
print('Mapping  reactions')
df_react_map = map_ids_to_db(df_model_id, ec_code ,'metanetx.reaction', 'metanetx.reaction', 'ec-code', ',')
print('ec_code reactions \n',df_react_map)

# 5. add ec-codes to model and Memote test
print('adding ec-codes to the model')
mnx_react = add_ids_to_model(mitocore, df_react_map, 'reactions','ec-code')


6.3.1.2 is  <class 'str'>
['1.4.1.13', '1.4.1.14', '1.4.7.1', '2.4.2.14', '3.5.1.2', '3.5.1.38', '4.3.3.6', '6.3.4.2', '6.3.5.2', '6.3.5.4', '6.3.5.5', '6.3.5.7'] is  <class 'list'>
['1.2.1.19', '1.2.1.21', '1.2.1.3', '1.2.1.4', '1.2.1.5', '1.2.1.69'] is  <class 'list'>
5.5.1.19 is  <class 'str'>
2.1.2.10 is  <class 'str'>
1.4.4.2 is  <class 'str'>
['1.6.2.6', '1.8.1.4'] is  <class 'list'>
6.3.2.2 is  <class 'str'>
['1.2.1.3', '1.2.1.54'] is  <class 'list'>
['1.4.1.13', '1.4.1.3', '1.4.1.4'] is  <class 'list'>
1.8.1.7 is  <class 'str'>
['1.6.4.2', '1.8.1.7', '1.8.4.2'] is  <class 'list'>
['3.6.1.11', '3.6.1.40'] is  <class 'list'>
1.2.1.99 is  <class 'str'>
3.5.1.94 is  <class 'str'>
2.5.1.41 is  <class 'str'>
3.1.3.69 is  <class 'str'>
['2.3.2.2', '3.4.19.14'] is  <class 'list'>
[] is  <class 'list'>
['2.3.2.2', '3.4.19.13'] is  <class 'list'>
[] is  <class 'list'>
4.1.2.20 is  <class 'str'>
3.6.3.17 is  <class 'str'>
1.4.7.1 is  <class 'str'>
6.3.1.2 is  <class 'str'>
['3.6.3.21', '7

In [ ]:
#cobra.io.write_sbml_model(mitocore, "Mitocore_automat_react.xml")
#!memote report snapshot --filename "Mitocore_automat_react.html" Mitocore_automat_react.xml

The current solver interface glpk doesn't support setting the optimality tolerance.
============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/benjaminreyes
plugins: anyio-3.5.0, typeguard-4.4.2
collected 155 items / 1 skipped                                                

../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF.FFFFFFF.FF.F.FF.FFF.FFFFFFFF..         [ 41%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 42%]
...F.FF.......F.F.F.FF                                                   [ 56%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 57%]
FFFFFF....FFFFF.FF                                                       [ 69%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_consistency.

# ------------GENES CHECKPOINT-------------


# Update gene annotations

After alignment of GPR list to Human1 new genes were added. This genes require Uniprot accesions for AutoPACMEN.The new entries for the genes have as 'id' Ensembl identifiers, which are used in Human1 too 

# Add Human1 identifiers

The new entries for the genes have as 'id' Ensembl identifiers, which are used in Human1 too 

Worflow:
1. Load latest SBML model
2. transfer Human1 ids from '_id' to '_annotations'
3. Save new model

In [40]:
# 2. transfer Human1 ids from '_id' to '_annotations'
# collect list for Human1 ids (Ensembl ids)
Human1_ids=[]

#loop over genes in mitocore
for gene in mitocore.genes:
    #get Human1 id if existing otherwise 'None' entry
    human1_id= gene.annotation.get('Human1', None)
    # use gene id as Human1 id if no Human1 id existing
    id=gene.id
    Human1_ids.append(id)

    if human1_id is None:
        gene.annotation['Human1'] = id


# Add Uniprot identifiers

The new entries for the genes have as 'id' Ensembl identifiers, which are used in Human1 too 

Worflow:

2. Extract Human1 gene ids from model
3. Map Human1 gene ids to uniprot ids
4. Add Uniprot ids, and run Memote test 

In [41]:

# 2. extract Human1 gene ids from model
human1_ids= get_model_ids_for_databases(mitocore, 'genes', '_annotation', ['Human1'])

#3. map Human1 gene ids to uniprot ids
human1_genes= pd.read_csv('Files/genes.tsv', sep='\t')
df_uniprot_genes = map_ids_to_db(human1_ids, human1_genes,'Human1', 'genes','geneUniProtID', '\t')

print('human1 metabolites \n',df_uniprot_genes)
# rename columns to match naming convention
df_uniprot_genes = df_uniprot_genes.rename(columns={'model_id': 'model_id', 'geneUniProtID': 'uniprot'})
print(df_uniprot_genes)

# 4. add uniprot ids to model and Memote test
Uniprot_genes= add_ids_to_model(mitocore, df_uniprot_genes, 'genes', 'uniprot')


Match found: ENSG00000156515 -> P19367
Match found: ENSG00000159399 -> P52789
Match found: ENSG00000160883 -> P52790
Match found: ENSG00000106633 -> P35557
Match found: ENSG00000131482 -> P35575
Match found: ENSG00000105220 -> P06744
Match found: ENSG00000152556 -> P08237
Match found: ENSG00000130957 -> O00757
Match found: ENSG00000165140 -> P09467
Match found: ENSG00000149925 -> P04075
Match found: ENSG00000109107 -> P09972
Match found: ENSG00000111669 -> P60174
Match found: ENSG00000111640 -> P04406
Match found: ENSG00000102144 -> P00558
Match found: ENSG00000171314 -> P18669
Match found: ENSG00000164708 -> P15259
Match found: ENSG00000111674 -> P09104
Match found: ENSG00000074800 -> P06733
Match found: ENSG00000108515 -> P13929
Match found: ENSG00000067225 -> P14618
Match found: ENSG00000124253 -> P35558
Match found: ENSG00000111716 -> P07195
Match found: ENSG00000160211 -> P11413
Match found: ENSG00000130313 -> O95336
Match found: ENSG00000142657 -> P52209
Match found: ENSG00000153

# Indirect accesion of gene species specific KEGG identifiers (API requests)


The KEGG orthology cannot be added directly by API request. Since Mitocore is a Human model, it requires specific Homo-sapiens identifiers (hsa) to send an API request to the KEGG database to retrieve KEGG orthology.

Note: The results of the API request takes long therefore is saves as csv file  (don't repeat it, number of genes shouldn't have changed)


Worflow:
2. Extract uniprot gene ids from model
3. API request to kegg to obtain hsa ids from uniprot ids
4. Save csv file with hsa and uniprot ids 
5. API request to KEGG to obtain KEGG ids from hsa ids

In [ ]:


# 2. extract uniprot gene ids from model
uniprot_ids= get_model_ids_for_databases(mitocore, 'genes', '_annotation', ['uniprot'])

# 3. API request to kegg to obtain hsa ids 
import pandas as pd
import time
from urllib import request
# ensure no whitespace or version suffixes
uniprot_ids['uniprot'] = uniprot_ids['uniprot'].astype(str).str.strip()

output_dict = {'uniprot': [], 'hsa': []}
batch_size = 10
uniprot_ids = list(uniprot_ids['uniprot'])

for i in range(0, len(uniprot_ids), batch_size):
    batch = uniprot_ids[i:i + batch_size]
    url = "https://rest.kegg.jp/conv/hsa/" + "+".join([f"uniprot:{uid}" for uid in batch]) #/conv/genes/

    try:
        with request.urlopen(url) as f:
            response = f.read()

        lines = response.decode('utf-8').splitlines()
        print(f"Requested {len(batch)} IDs; got {len(lines)} mappings")

        if lines:
            for line in lines:
                if '\t' in line:
                    uniprot_entry, hsa_id = line.strip().split('\t')
                    uniprot = uniprot_entry.replace("up:", "").strip()
                    hsa = hsa_id.replace("hsa:", "").strip()

                    output_dict['uniprot'].append(uniprot)
                    output_dict['hsa'].append(hsa)

    except Exception as e:
        print(f"Failed on batch starting with {batch[0]}: {e}")
        for uid in batch:
            output_dict['uniprot'].append(uid)
            output_dict['hsa'].append(None)

    time.sleep(3)

# 4.create dataframe with uniprot and hsa ids and save as csv file
hsa_ids_Genes = pd.DataFrame.from_dict(output_dict)
print(f"Final DataFrame has {len(hsa_ids_Genes)} entries")
print(hsa_ids_Genes)



Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 11 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 11 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 IDs; got 10 mappings
Requested 10 I

In [ ]:
# 5. API request to kegg to obtain kegg gene ids for hsa ids
import pandas as pd
from urllib import request

# Download the full KEGG gene-to-KO mapping for human
url = "http://rest.kegg.jp/link/ko/hsa"
response = request.urlopen(url)
response_content = response.read().decode('utf-8')  # Decode the response content

# Parse the mapping
mapping = []
for line in response_content.strip().split("\n"):
    gene_id, ko_id = line.split("\t")
    mapping.append((gene_id, ko_id.split(":")[1]))  # remove 'ko:' prefix

# Convert to DataFrame
mapping_df = pd.DataFrame(mapping, columns=["hsa", "kegg.genes"])
print(mapping_df)

# Match MIRIAM pattern for kegg.genes
for index, row in mapping_df.iterrows():
    kegg_id = row["kegg.genes"]
    if not kegg_id.startswith("hsa:"):  # Check if prefix is missing
        kegg_id = f"hsa:{kegg_id}"  # Add prefix
        mapping_df.at[index, 'kegg.genes'] = kegg_id
    hsa_id = row["hsa"] 
    if kegg_id.startswith("hsa:"):
        hsa_id = row["hsa"].split(":")[1]
        mapping_df.at[index, 'hsa'] = hsa_id


                 hsa kegg.genes
0             hsa:10     K00622
1            hsa:100     K01488
2           hsa:1000     K06736
3          hsa:10000     K04456
4      hsa:100008587     K01986
...              ...        ...
18947       hsa:9991     K17844
18948       hsa:9992     K04896
18949       hsa:9993     K27941
18950       hsa:9994     K25593
18951       hsa:9997     K23755

[18952 rows x 2 columns]


# Add hsa and KEGG identifiers to the model 

After API request to obtain the hsa orthology and subsequently KEGG identifiers 

Worflow:
2. Extract Uniprot gene ids from model
3. Map uniprot ids to hsa ids
4. Add hsa ids

repeat workflow for KEGG replacing 'Uniprot' through hsa and and run Memote test.

In [45]:
# hsa orthology for genes
# 2. extract uniprot gene ids from model
print('geting ids for all uniprot')
df_genes = get_model_ids_for_databases(mitocore, 'genes', '_annotation', ['uniprot'])
print(df_genes)

# 3. map uniprot ids to hsa ids 
print('Mapping  metabolites to hsa')
df_hsa_genes = map_ids_to_db(df_genes, hsa_ids_Genes,'uniprot', 'uniprot', 'hsa', ',')
print('hsa metabolites \n',df_hsa_genes)

# 4. add hsa ids to model
print('adding hsa ids to the model')
hsa_react = add_ids_to_model(mitocore, df_hsa_genes, 'genes','hsa')

geting ids for all uniprot
            model_id uniprot
0    ENSG00000156515  P19367
1    ENSG00000159399  P52789
2    ENSG00000160883  P52790
3    ENSG00000106633  P35557
4    ENSG00000131482  P35575
..               ...     ...
472  ENSG00000164363  Q96N87
473  ENSG00000112077  Q02094
474  ENSG00000138449  Q9NP59
475  ENSG00000018280  P49279
476  ENSG00000110911  P49281

[477 rows x 2 columns]
Mapping  metabolites to hsa
No match found for P19367, using default value None
No match found for P52789, using default value None
No match found for P52790, using default value None
No match found for P35557, using default value None
No match found for P35575, using default value None
No match found for P06744, using default value None
No match found for P08237, using default value None
No match found for O00757, using default value None
No match found for P09467, using default value None
No match found for P04075, using default value None
No match found for P09972, using default value None
N

In [46]:
# KEGG identifiers for genes

# 2. extract hsa gene orthology from model
print('geting ids for all uniprot')
df_genes = get_model_ids_for_databases(mitocore, 'genes', '_annotation', ['hsa'])
print(df_genes)

# 3. map hsa orthology to kegg ids 
print('Mapping  metabolites to hsa')
df_kegg_genes = map_ids_to_db(df_genes, mapping_df,'hsa', 'hsa', 'kegg.genes', ',')
print('hsa metabolites \n',df_kegg_genes)

# 4. add hsa ids to model
print('adding kegg ids to the model')
kegg_genes = add_ids_to_model(mitocore, df_kegg_genes, 'genes','kegg.genes')



geting ids for all uniprot
            model_id   hsa
0    ENSG00000156515  None
1    ENSG00000159399  None
2    ENSG00000160883  None
3    ENSG00000106633  None
4    ENSG00000131482  None
..               ...   ...
472  ENSG00000164363  None
473  ENSG00000112077  None
474  ENSG00000138449  None
475  ENSG00000018280  None
476  ENSG00000110911  None

[477 rows x 2 columns]
Mapping  metabolites to hsa
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value None
No match found for None, using default value No

In [ ]:
# load and clean model from N/A values
mitocore = clean_invalid_annotations(mitocore)


In [ ]:
# Save and Memote report 
#cobra.io.write_sbml_model(mitocore, "Mitocore_autom_genes.xml")
#!memote report snapshot --filename "Mitocore_autom_genes.html" Mitocore_autom_genes.xml

The current solver interface glpk doesn't support setting the optimality tolerance.
============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/benjaminreyes
plugins: anyio-3.5.0, typeguard-4.4.2
collected 155 items / 1 skipped                                                

../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py F [  0%]
F.FFFFFFFFFFFFFFFFFFFFF.FFFFFFFFF.FFFFFFF.FF.F.FF.FFF.FFFFFFFF..         [ 41%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 42%]
...F.FF.......F.F.F.FF                                                   [ 56%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 57%]
FFFFFF....FFFFF.FF                                                       [ 69%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_consistency.

# ---------METABOLITES CHECKPOINT-----------


In [49]:
# 1. load MNX database file for metabolites
mapping_kegg= mnx_processing_file_mnx('Files/chem_xref.tsv','kegg.compound','metabolites',( "kegg.compound:", "keggC:"))
mapping_bigg= mnx_processing_file_mnx('Files/chem_xref.tsv','bigg.metabolite','metabolites',("bigg.metabolite:", "biggM:"))

# 2. merge files into one mapping file
metanetx_mapp_metabolites = pd.merge(mapping_bigg, mapping_kegg, on='metanetx.metabolites', how='outer')
print(metanetx_mapp_metabolites)
#rename columns in dataframe to match MIRIAM pattern
metanetx_mapp_metabolites = metanetx_mapp_metabolites.rename(columns={'bigg.metabolite': 'bigg.metabolite', 'metanetx.metabolites': 'metanetx.chemical', 'kegg.compound': 'kegg.compound'})
print(metanetx_mapp_metabolites)


      bigg.metabolite metanetx.metabolites kegg.compound
0                 oh1               MNXM02        C01328
1                 oh1               MNXM02        C01328
2                 oh1               MNXM02      M_C01328
3               M_oh1               MNXM02        C01328
4               M_oh1               MNXM02        C01328
...               ...                  ...           ...
90571           M_h2o                WATER        C00001
90572           M_h2o                WATER      M_C00001
90573             h2o                WATER        C00001
90574             h2o                WATER        C00001
90575             h2o                WATER      M_C00001

[90576 rows x 3 columns]
      bigg.metabolite metanetx.chemical kegg.compound
0                 oh1            MNXM02        C01328
1                 oh1            MNXM02        C01328
2                 oh1            MNXM02      M_C01328
3               M_oh1            MNXM02        C01328
4               M_oh

workflow add Human1 and MetaNetX identifiers:

2. Extract KEGG and BiGG metabolite ids from model into dataframe
3. Map KEGG ids to Human1 
4. Add Human1 ids and save model

repeat for MetaNetx ids with Metabolites_metanetx mapping file and Metanetx.chemical instead Human1 and run Memote test

In [50]:

# 2. extract kegg and bigg ids from model into dataframe
print('geting ids for all possible databases ')
df_metab = get_model_ids_for_databases(mitocore, 'metabolites', '_annotation', ['kegg', 'bigg'])
print(df_metab)

# 3. map kegg ids to Human1 metabolites mapping file
print('Mapping metabolites to Human1 ids.')
human1_mets= pd.read_csv('Files/metabolites.tsv', sep='\t')
df_Human1_metab = map_ids_to_db(df_metab, human1_mets,'kegg', 'metKEGGID', 'mets','\t')
print(df_Human1_metab)
#rename columns to match naming convention
df_Human1_metab = df_Human1_metab.rename(columns={'model_id': 'model_id', 'mets': 'Human1'})
print(df_Human1_metab)

# add Human1 ids to model
print('adding Human1 ids to the model')
add_ids_to_model(mitocore,df_Human1_metab,  'metabolites','Human1' )

geting ids for all possible databases 
       model_id    kegg        bigg
0      10fthf_c  C00234      10fthf
1      10fthf_m  C00234      10fthf
2       13dpg_c  C00236       13dpg
3    1pipdn2c_c  C04092    1pipdn2c
4      1pyr5c_m  C03912        None
..          ...     ...         ...
436   myrsACP_c  C05761        None
437   HC01605_c  C05762   3opalmACP
438   HC01326_c  C04633   3hpalmACP
439   HC01606_c  C05763  tpalm2eACP
440   palmACP_c  C05764        None

[441 rows x 3 columns]
Mapping metabolites to Human1 ids.
Match found: C00234 -> MAM00266c
Match found: C00234 -> MAM00266c
Match found: C00236 -> MAM00247c
Match found: C04092 -> MAM01663x
Match found: C03912 -> MAM00559c
Match found: C03508 -> MAM02321m
Match found: C05984 -> MAM00648c
Match found: C05984 -> MAM00648c
Match found: C03344 -> MAM00662m
Match found: C03345 -> MAM02999m
Match found: C01033 -> MAM00663m
Match found: C00349 -> MAM00661c
Match found: C03460 -> MAM02468m
Match found: C00109 -> MAM00671c
Match fo

Name,S1SBMLmodel
Memory address,165692810
Number of metabolites,441
Number of reactions,555
Number of genes,477
Number of groups,96
Objective expression,1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
Compartments,"Cytosol, Mitochondrion, External"


In [51]:

# 2. extract kegg and bigg ids from model into dataframe
print('geting ids for all possible databases ')
df_metab = get_model_ids_for_databases(mitocore, 'metabolites', '_annotation', ['kegg.compound', 'bigg.metabolite'])
print(df_metab)

# 3. map kegg ids to metanetx ids metabolites mapping file
print('Mapping  metabolites to metanetx.')
df_MNX_metab = map_ids_to_db(df_metab, metanetx_mapp_metabolites,'kegg.compound', 'kegg.compound', 'metanetx.chemical', ',')
print('MNX metabolites \n',df_MNX_metab)

# add metanetx ids to model
print('adding MNX ids to the model')
MNX_metab = add_ids_to_model(mitocore, df_MNX_metab,  'metabolites','metanetx.chemical' )


geting ids for all possible databases 
       model_id kegg.compound bigg.metabolite
0      10fthf_c        C00234          10fthf
1      10fthf_m        C00234          10fthf
2       13dpg_c        C00236           13dpg
3    1pipdn2c_c        C04092        1pipdn2c
4      1pyr5c_m        C03912            None
..          ...           ...             ...
436   myrsACP_c        C05761            None
437   HC01605_c        C05762       3opalmACP
438   HC01326_c        C04633       3hpalmACP
439   HC01606_c        C05763      tpalm2eACP
440   palmACP_c        C05764            None

[441 rows x 3 columns]
Mapping  metabolites to metanetx.
Match found: C00234 -> MNXM1102376
Match found: C00234 -> MNXM1102376
Match found: C00236 -> MNXM1108073
Match found: C04092 -> MNXM911
Match found: C03912 -> MNXM1105060
Match found: C03508 -> MNXM114087
Match found: C05984 -> MNXM1103784
Match found: C05984 -> MNXM1103784
Match found: C03344 -> MNXM1104585
Match found: C03345 -> MNXM1104649
Match 

# Final cleanning of the Human1 aligned model from N/A values 

cleans empty entries of None, N/A, and '' values, this values are account into memote as positive entries however these are not valid identifiers

In [ ]:
# Clean N/A keys added during annotation curation in previous steps
mitocore = clean_invalid_annotations(mitocore)

# save cleaned model as new SBML file
cobra.io.write_sbml_model(mitocore, "Mitocore.xml")

# Run memote snapshot on the corrected model
!memote report snapshot --filename "Mitocore.html" Mitocore.xml #"Mitocore_automat_mets.html" Mitocore_automat_mets.xml
# ---End---

The current solver interface glpk doesn't support setting the optimality tolerance.
============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/benjaminreyes
plugins: anyio-3.5.0, typeguard-4.4.2
collected 155 items / 1 skipped                                                

../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py F [  0%]
F.FFFFFFFFFFFFFFFFFFFFF.FFFFFFFFF.FFFFFFF.FF.F.FF.FFF.FFFFFFFF..         [ 41%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 42%]
...F.FF.......F.F.F.FF                                                   [ 56%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 57%]
FFFFFF....FFFFF.FF                                                       [ 69%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_consistency.

In [53]:
# Run memote snapshot on the corrected model
!memote report snapshot --filename "Mitocore_automat_mets_1.html" Mitocore_automat_mets.xml

The current solver interface glpk doesn't support setting the optimality tolerance.
============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/benjaminreyes
plugins: anyio-3.5.0, typeguard-4.4.2
collected 155 items / 1 skipped                                                

../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_annotation.py F [  0%]
F.FFFFFFFFFFFFFFFFFFFFF.FFFFFFFFF.FFFFFFF.FF.F.FF.FFF.FFFFFFFF..         [ 41%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_basic.py . [ 42%]
...F.FF.......F.F.F.FF                                                   [ 56%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_biomass.py . [ 57%]
FFFFFF....FFFFF.FF                                                       [ 69%]
../../../../../anaconda3/lib/python3.10/site-packages/memote/suite/tests/test_consistency.